## Entraînement du modèle DIST_REL_C_RNN_01

Les modèles DELTA_DIST+H tentant de prédire les distances convergées de tous les atomes d'une molécule à partir des distances bruitées des atomes à des points du repère donnant de mauvais résultats, nous tentons une nouvelle approche avec un problème plus simple, pour essayer de valider la capacité des modèles à prédire des distances entre des atomes. Ce modèle ne sera toutefois pas utilisable en situations réelles (nous ne pourrons pas reconstruire les positions des atomes à partir des prédictions).

Le modèle DIST_REL_C_RNN a pour objectif de prédire la distance entre deux atomes deux carbones partageant une liaison à partir des informations suivantes pour chaque autre atome de la molécule : 

* La distance avec les deux atomes de carbone
* La classe positionnelle de l'atome par rapport aux deux atomes de carbones. Pour la déterminer, on compare la  position de l'atome par rapport aux deux plans P1 (resp. P2) de vecteur normal C1C2 et passant par point C1 (resp. C2). Si l'atome est entre les deux plans il aura pour classe 'C', et s'il est derrière le plan P1 (resp. P2) il aura pour classe 'G' (resp 'D').
* Le numéro atomique de l'atome (encodé en one hot encoding)
* La masse atomique de l'atome.

Contrairement aux modèles précédents qui étaient des réseaux profonds entièrement connectés, nous allons ici créer un réseau récurrent, prenant tour à tour les informations de chaque atome de la molécule (hors les deux atomes de carbones) et devant prédire à la fin du cycle la distance entre les deux atomes de carbones.

Nous allons dans ce notebook entraîner une première version de ce modèle, avec des paramètres choisis naïvement, puis nous tenterons par la suite d'optimiser le modèle.

#### Chemin des fichiers

In [16]:
minimal_prepared_input_loc = "../data/DIST_REL_C_RNN/minimal_prepared_input.h5"
minimal_labels_loc = "../data/DIST_REL_C_RNN/minimal_labels.h5"

train_prepared_input_loc = "../data/DIST_REL_C_RNN/train_set_prepared_input.h5"
train_labels_loc = "../data/DIST_REL_C_RNN/train_set_labels.h5"

models_loc = "../models/DIST_REL_C_RNN_01/9.2/"
logs_loc = "../models/DIST_REL_C_RNN_01/9.2/"

### Création du RN

#### Fonction de coût (RMSE)

In [23]:
def rmse(pred, targets):
    with tf.name_scope("rmse_loss"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, targets)), name="rmse")

#### Fonction d'évaluation des performances (opposé du RMSE)

In [1]:
def rmse_valid(pred, targets, inputs):
    with tf.name_scope("rmse_validation"):
        return -rmse(pred, targets)

#### Fonction de création du RN

Cette fonction est largement inspirée de l'exemple suivant : https://github.com/tflearn/tflearn/blob/master/examples/images/rnn_pixels.py

In [25]:
import tflearn as tfl


def creer_RNN(batch_size, activation="tanh", inner_activation="sigmoid", outlayer_activation="linear",
              depth=2, width=58, dropout_val=0.98, learning_rate=0.001):
    
    # Nombre d'atomes par exemple indéfini [1, 58] -> None
    # 15 valeurs par atome
    network = tfl.input_data(shape=[None, 58, 15], name="input")
    
    for i in range(depth):
        network = tfl.lstm(network, width, return_seq=(i != depth-1), name="hl"+str(i))
        network = tfl.dropout(network, dropout_val)
        
    network = tfl.fully_connected(network, 1, activation='linear')
    
    network = tfl.regression(network, optimizer='adam', loss=rmse, metric=rmse_valid, name="outlayer", 
                             learning_rate=learning_rate)
    
    return network
    

## Entraînement du modèle

In [26]:
import tensorflow as tf

def train_DIST_REL_C_RNN(X, y, model_name, model_path, logs_path, batch_size=1000, epochs=2, 
                         learning_rate=0.001, dropout_val=0.98, act="tanh", inner_act="sigmoid",
                         outlayer_act="linear", depth=2, width=58):
    
    tf.reset_default_graph()
        
    # Création du réseau
    network = creer_RNN(batch_size, activation=act, inner_activation=inner_act, outlayer_activation=outlayer_act,
                        depth=depth, width=width, dropout_val=dropout_val, learning_rate=learning_rate)
    
    # Création du modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)
    
    # Entraînement
    model.fit(X_inputs=X,Y_targets=y, batch_size=batch_size,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name, n_epoch=epochs)
    
    
    # Sauvegarde du modèle
    model.save(model_path+model_name+".tfl")

    


In [29]:
import h5py
import numpy as np
import tensorflow as tf
from tflearn.data_utils import pad_sequences


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')
"""
input_X_h5 = h5py.File(minimal_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(minimal_labels_loc, 'r')
"""

input_X = np.array(input_X_h5["inputs"])

input_X = pad_sequences(input_X, dtype="float32", maxlen=870)

input_X = input_X.reshape(-1, 58, 15)

labels_y = np.array(labels_y_h5["targets"])
labels_y = labels_y.reshape(-1, 1)


model_name = "DIST_REL_C_RNN_01_basic"

model_path = models_loc
logs_path = logs_loc

train_DIST_REL_C_RNN(input_X, labels_y, model_name, model_path, logs_path, batch_size=5000, 
                     epochs=100, learning_rate=0.001, dropout_val=0.98, depth=1, width=58)


---------------------------------
Run id: DIST_REL_C_RNN_01_basic
Log directory: ../models/DIST_REL_C_RNN_01/9.2/
INFO:tensorflow:Summary name rmse_validation/ (raw) is illegal; using rmse_validation/__raw_ instead.
---------------------------------
Training samples: 0
Validation samples: 0
--


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/etudiant/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/etudiant/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/tflearn/data_flow.py", line 201, in fill_batch_ids_queue
    ids = self.next_batch_ids()
  File "/home/etudiant/anaconda3/lib/python3.6/site-packages/tflearn/data_flow.py", line 215, in next_batch_ids
    batch_start, batch_end = self.batches[self.batch_index]
IndexError: list index out of range

